### hole-hole extended RPA

wavefunction:

$|\Psi^{(N-2)}_{\lambda}> = \sum_{pq} c_{pq,\lambda} a_q a_p |\Psi^{N}>$

##### GEVP to be solved

Option 1: label $\textbf{hh}$

$\sum_{pq}c_{pq,\lambda} <\Psi^N|[a^\dagger_r a^\dagger_s,[H,a_q a_p]]|\Psi^N> = \Delta E_{\lambda} \sum_{pq}c_{pq,\lambda} <\Psi^N|[a^\dagger_r a^\dagger_s,a_q a_p]|\Psi^N>$

$A_{rs,pq} = <\Psi|[a^\dagger_r a^\dagger_s,[H,a_q a_p]]|\Psi>$

$M_{rs,pq} = <\Psi|[a^\dagger_r a^\dagger_s, a_q a_p]|\Psi>$

Option 2: label $\textbf{hh2}$

$\sum_{pq}c_{pq,\lambda} <\Psi^N|[a^\dagger_r a^\dagger_s,[H,a_q a_p]]|\Psi^N> = \Delta E_{\lambda} \sum_{pq}c_{pq,\lambda} <\Psi^N|a^\dagger_r a^\dagger_s a_q a_p |\Psi^N>$

$A_{rs,pq} = <\Psi|[a^\dagger_r a^\dagger_s,[H,a_q a_p]]|\Psi>$

$M_{rs,pq} = <\Psi^N|a^\dagger_r a^\dagger_s a_q a_p |\Psi^N>$

<!-- Overlap

$M_{rs,pq} = <\Psi|a^\dagger_r a^\dagger_s a_q a_p|\Psi>$

$C^T_{rs} M_{rs,pq} C_{pq} =? I$ -->

##### Eigenvalue problem solvers

Starting from the notation

$A C = w M C$

Option 1: label $\textbf{asymm}$

$ M^{-1} A C = w C$

comment: should be applicable to both GEVP forms, as long as $M$ is invertible.

Option 2: label $\textbf{lowdin}$

$M^{-1/2} A M^{-1/2} M^{1/2} C = w M^{-1/2} M M^{-1/2} M^{1/2} C$

$A^{'} C^{'} = w C^{'}$

comment: only applicable to GEVP form $\textbf{hh2}$, for which $M$ should be positive definite.

Here I took both hh-ERPA formulations (labeled as $\textbf{hh}$ and $\textbf{hh2}$) combined with the asymmetric and symmetric solvers (labeled $\textbf{asymm}$ and $\textbf{lowdin}$, respecively) and compare results in terms of spectrum ($\Delta E_{\lambda} > 0$), trace of reconstructed 2-RDM and electronic energy value. 

Given that the symmetric solver is not suitable for the $\textbf{hh}$ GEVP, as the metric matrix $M$ is indefinite, I went around by setting to zero $M$'s negative eigenvalues.

The new 2-RDM and the energy were computed as:

$\Gamma_{ijkl} = <\Psi^N|a^\dagger_i a^\dagger_j a_l a_k |\Psi^N>$

$\Gamma_{ijkl} = \sum_{n} <\Psi^N|a^\dagger_i a^\dagger_j|\Psi^{(N-2)}_{n}><\Psi^{(N-2)}_{n}| a_l a_k |\Psi^N>$

where

$<\Psi^{(N-2)}_{n}| a_l a_k |\Psi^N> \approx \sum_{pq}c_{pq,n} <\Psi^N|a^\dagger_p a^\dagger_q a_l a_k |\Psi^N>$

$<\Psi^{(N-2)}_{n}| a_l a_k |\Psi^N> \approx \sum_{pq}c_{pq,n} \Gamma_{pqkl}$


This approximated $\Gamma_{ijkl}$ I'll refer to as $\Gamma^{RPA}_{ijkl}$

$E^{RPA}_0 = \sum^{m}_{pq} h_{pq} \gamma_{pq} + \frac{1}{4} \sum_{pqrs} <pq||rs> \Gamma^{RPA}_{pqrs}$

In [1]:
import sys  
sys.path.insert(0, '../')

In [2]:
import os
import numpy as np
np.set_printoptions(precision=4)
import pandas as pd
from iodata import load_one
import pyci

In [3]:
# Import modules
from src.scripts.script_erpa import build_gevp
from src.scripts.tools_erpa import get_folder_path
# from src.scripts.rhotransition import reconstruct_dm2
from eomee.tools import pickpositiveeig, pickeig, spinize, antisymmetrize, hartreefock_rdms

Define test cases/systems

In [4]:
def get_npz(folder, method, basis, job_name, label):
    dir_path = get_folder_path(folder, job_name, method, basis, charge=0, mult=1, state=0)
    if not os.path.isdir(dir_path):
        raise AssertionError(f"Directory {dir_path} does not exist!")
    
    prefix = os.path.basename(folder).split("_")[0] # ID of molecule
    basisname = basis.lower().replace("-", "").replace("*", "p").replace("+", "d")
    fname = f"{prefix}_{job_name.lower()}_{method.lower()}_{basisname}.{label}"
    npz_path = f'{dir_path}/{fname}.npz'
    if not os.path.isfile(npz_path):
        raise AssertionError(f"File {npz_path} does not exist!")

    return os.path.abspath(npz_path)


def get_file(folder, method, basis, job_name, ext):
    dir_path = get_folder_path(folder, job_name, method, basis, charge=0, mult=1, state=0)
    if not os.path.isdir(dir_path):
        raise AssertionError(f"Directory {dir_path} does not exist!")
    
    prefix = os.path.basename(folder).split("_")[0] # ID of molecule
    basisname = basis.lower().replace("-", "").replace("*", "p").replace("+", "d")
    fname = f"{prefix}_{job_name.lower()}_{method.lower()}_{basisname}"
    mol_path = f'{dir_path}/{fname}.{ext}'
    if not os.path.isfile(mol_path):
        raise AssertionError(f"File {mol_path} does not exist!")

    return os.path.abspath(mol_path)


def wrap_pyci(nparts, fname, ref):
    import pyci
    wfntype = {'doci': pyci.doci_wfn, 'fci': pyci.fullci_wfn, }
    # ham = pyci.hamiltonian(nuc_rep, h, g)
    ham = pyci.hamiltonian(fname)
    if ref == 'rhf':
        wfn = pyci.fullci_wfn(ham.nbasis, *nparts)
        wfn.add_hartreefock_det()
    elif ref in ['doci', 'fci']:
        wfn = wfntype[ref](ham.nbasis, *nparts)
        wfn.add_all_dets()
    else:
        raise ValueError('wrong wavefunction type.')
    op = pyci.sparse_op(ham, wfn)
    ev = op.solve(n=1, tol=1.0e-9)[0]
    return ev[0]

In [5]:
def compile_table(_folder, _method, _basis, operator, solvers, eps=1.0e-4):
    op = operator
    results = []

    for slvr in solvers:
        label = op + slvr + 'trnct'
        fnpz = get_npz(_folder, _method, _basis, 'sp', label)
        data = np.load(f"{fnpz}")
        es = data['energy']
        excs = np.sort(es[es.real > eps].real) #pickeig(es, tol=eps)  # take only unique values #
        results.append(excs)
    return results


def get_unique_nvals(operator, solvers, results, nvals=3, tol=1.0e-3):
    output = {}
    op = operator
    for i, slvr in enumerate(solvers):
        label = op + slvr
        es = results[i]
        exc = pickeig(es, tol=tol)  # take only unique values
        exc = tuple(exc)[:nvals]
        temp = np.array([0.,]*nvals )   
        temp[:len(exc)] = exc    
        output[label] = temp
    return output
    

In [6]:
folder = 'be'
method = 'doci'
basis = 'aug-cc-pvdz' # 'sto-6g' #  

In [7]:
molden = get_file(folder, method, basis, 'sp', 'molden')
fcidump = get_file(folder, method, basis, 'sp', 'FCIDUMP')
mol = load_one(f'{molden}')
n_up = int(mol.nelec //2)
nparts = (n_up, n_up)

# e_occs = np.zeros(n_vals)
# e_occs[:n_up] = sorted(-mol.mo.energies[:n_up])
# df['Occ MOs'] = e_occs

# Take pair orbital energies as reference for double IP values:
dip = []
e_occs = mol.mo.energies[:n_up]  # occupied MO energies (closed shell, restricted SCF case, e_mos_a = e_mos_b)
# truncate number of DIP values to expect
n_vals = n_up*(n_up-1)//2       # alpha alpha terms, only upper triangle
n_vals *= 2                     # alpha beta terms, only upper triengle
n_vals += n_up                  # alpha beta diagonal
# getting two-electron integrals
ham = pyci.hamiltonian(fcidump)
k = ham.nbasis
twomo = spinize(ham.two_mo)
two_mo = antisymmetrize(twomo)

# pair_orbt_e = - (e_i + e_j - <ij||ij>)
for i in range(n_up):
    for j in range(i+1,n_up):
        # alpha alpha DIP
        value = e_occs[i] + e_occs[j] - two_mo[i,j,i,j]
        dip.append(-value)
    for j in range(i+1,n_up):
        # alpha beta DIP
        value = e_occs[i] + e_occs[j] - two_mo[i,j+k,i,j+k]
        dip.append(-value)
    # alpha beta DIP
    value = 2 * e_occs[i] - two_mo[i,i+k,i,i+k]
    dip.append(-value)

In [8]:
transition = 'hh'
epsolvers = ['asymm', 'symmsvd', 'lowdin', 'lowdinimg']

# n_vals = 4 # 15 # 
# outs = compile_table(folder, method, basis, transition, epsolvers)
# results = get_unique_nvals(transition, epsolvers, outs, nvals=n_vals, tol=1.0e-3)
# df = pd.DataFrame(results)

In [9]:
# df['DIP'] = sorted(dip)

In [10]:
# df

In [11]:
folder = 'be'
method = 'doci'
basis = 'aug-cc-pvdz' # 'sto-6g' #  

transition = 'hh2'
epsolvers = ['asymm', 'symmsvd', 'lowdin', 'lowdinimg']

n_vals = 4 #15
# outs = compile_table(folder, method, basis, transition, epsolvers)
# results = get_unique_nvals(transition, epsolvers, outs, nvals=n_vals, tol=1.0e-3)

# df = pd.DataFrame(results)
# df['DIP'] = sorted(dip)


In [12]:
# df

In [13]:
import matplotlib.pyplot as plt


def plot_solvers_results(operator, results, nplots, solvers, min=0., max=1.0, n_bins=100, reference=None):
    assert len(results) == nplots
    # solvers = ['asymm', 'lowdin', 'lowdinimg']
    assert len(results) == len(solvers)
    HIST_BINS = np.linspace(min, max, n_bins)

    fig, axs = plt.subplots(1, nplots, sharey=True, tight_layout=True)

    for i in range(nplots):
        axs[i].hist(results[i], bins=HIST_BINS)
    if reference is not None:
        for i in range(nplots):
            axs[i].hist(reference, bins=HIST_BINS)

    for i, slvr in enumerate(solvers):
        axs[i].set_title(f'{operator}-{slvr}')

    for ax in axs.flat:
        ax.set(xlabel=f'$\omega$')

    plt.show()

In [14]:
# folder = 'be'
# basis = 'aug-cc-pvdz' # 'sto-6g' #
# solvers = ['asymm', 'lowdin', 'lowdinimg']
# n_plots = 3
# max=6.0
# wfn = 'rhf'

In [15]:
# op = 'hh'
# method = wfn
# results = compile_table(folder, method, basis, op, solvers)

# plot_solvers_results(op, results, n_plots, solvers, min=0., max=max, n_bins=100, reference=dip)

In [16]:
# op = 'hh2'
# method = wfn
# results = compile_table(folder, method, basis, op, solvers)

# plot_solvers_results(op, results, n_plots, solvers, min=0., max=max, n_bins=100, reference=dip)

$(\sum_n c^{EOM}_n c^{FCI}_n)^2$

[Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm)

$(Tr[1DM^{EOM} 1DM^{FCI}])^2$

Considerations when reconstructed the 2-RDM from the truncated GEVP solutions:

$\Gamma_{pq,rs} = <\Psi|a^\dagger_p a^\dagger_q a_s a_r|\Psi>$

Elements of symmetry:

$\Gamma_{pq,rs} = \Gamma_{qp,sr} = \Gamma_{rs,pq}  = \Gamma_{sr,qp}$

$\Gamma_{pq,rs} = -\Gamma_{pq,sr} = -\Gamma_{qp,rs}$

symmetry for spin-components:

$\Gamma^{aaaa}_{pq,rs} = \Gamma^{aaaa}_{qp,sr} = \Gamma^{aaaa}_{rs,pq}  = \Gamma^{aaaa}_{sr,qp}$

$\Gamma^{aaaa}_{pq,rs} = -\Gamma^{aaaa}_{pq,sr} = -\Gamma^{aaaa}_{qp,rs}$

$\Gamma^{abab}_{pq,rs} = \Gamma^{abab}_{rs,pq}$

$\Gamma^{abab}_{pq,rs} = \Gamma^{baba}_{qp,sr}$

ERPA $\Gamma$:

$\Gamma_{pq,rs} = <\Psi|a^\dagger_p a^\dagger_q a_s a_r|\Psi>\\ = \sum_n <\Psi|a^\dagger_p a^\dagger_q |\Psi^{N-2}_n><\Psi^{N-2}_n| a_s a_r|\Psi>\\ = \sum_n \gamma_{pq;n} \gamma_{rs;n}$

where $\gamma_{pq;n}$ is a transition density matrix element:

$\gamma_{pq;n} = <\Psi|a^\dagger_p a^\dagger_q |\Psi^{N-2}_n>\\ \approx \sum_{ij} c_{ij;n} <\Psi|a^\dagger_p a^\dagger_q a_j a_i|\Psi>\\
= \sum_{ij} c_{ij;n} M_{pq,ij}$

In [17]:
def reconstruct_dm2_truncated_gevp(coeffs, metricmx):
    from eomee.tools import antisymmetrize

    assert metricmx.ndim == 4
    n = metricmx.shape[0]
    n_pq = n*(n-1)//2
    metricmx = metricmx.reshape(n**2,n**2)
    M = np.zeros((n_pq, n_pq))
    configs = []
    assert coeffs.shape[1] == n_pq
    # Find p<q configurations and truncate metric matrix accordingly
    for i in range(n):
        for j in range(i+1,n):
            ij = n*i + j
            configs.append(ij)
    for i, rs in enumerate(configs):
        for j, pq in enumerate(configs):
            M[i,j] = metricmx[rs,pq]
    #
    # tdms_n = \sum_y c_y,n M_xy
    #
    tdms = np.einsum("ny,xy->nx", coeffs, M)
    # Fill terms Gamma_pq,rs --> Gamma_x,y
    # Gamma_x,y is symmetric with Gamma_pq,rs = Gamma_rs,pq
    temp = np.zeros((n_pq, n_pq))
    for rdm in tdms:
        temp += np.outer(rdm, rdm)
    # Unfold into 4-index tensor Gamma_p,q,r,s
    # Fill missing symmetric terms p>q, r>s
    dm2rpa = np.zeros((n,n,n,n))
    for x, pq in enumerate(configs):
        for y, rs in enumerate(configs):
            (p,q) = (pq // n, pq % n)
            (r,s) = (rs // n, rs % n)
            dm2rpa[p,q,r,s] = temp[x,y]
            dm2rpa[q,p,s,r] = temp[x,y]
    return antisymmetrize(dm2rpa)

In [18]:
folder = 'be'
method = 'rhf'
basis = 'aug-cc-pvdz' # 'sto-6g' # 

fcidump = get_file(folder, method, basis, 'sp', 'FCIDUMP')
ham = pyci.hamiltonian(fcidump)
k = ham.nbasis
ns = 2*k
nparts = (2, 2)

cinpz = get_npz(folder, 'fci', basis, 'sp', 'ci')
data = np.load(cinpz)
rdm1_fci, rdm2_fci = pyci.spinize_rdms(data['dm1'], data['dm2'])
tr_fci = np.trace(rdm1_fci @ rdm1_fci)

if method not in ['rhf', 'hf', 'uhf']:
    cinpz = get_npz(folder, method, basis, 'sp', 'ci')
    data = np.load(cinpz)
    rdm1, rdm2 = pyci.spinize_rdms(data['dm1'], data['dm2'])
else:
    rdm1, rdm2 = hartreefock_rdms(k, *nparts)


transition = 'hh2'
epsolvers = ['asymm', 'lowdin', 'lowdinimg']

In [19]:
results = {}
# gevp = build_gevp(transition, nparts, ham.one_mo, ham.two_mo, rdm1, rdm2, restricted=True)
metricmx = rdm2 # gevp.rhs.reshape(ns,ns,ns,ns) # 

for slvr in epsolvers:
    label = transition + slvr + 'trnct'
    fnpz = get_npz(folder, method, basis, 'sp', label)
    data = np.load(fnpz)
    es = data['energy']
    ev = data['coeff'] 
    coeffs= pickpositiveeig(es, ev)[1]
    dm2_rpa = reconstruct_dm2_truncated_gevp(coeffs, metricmx)
    #
    gamma = np.einsum('pqrq->pr', dm2_rpa) / (sum(nparts)-1)
    norm = np.trace(gamma) #np.einsum("ijij", dm2_rpa)/2
    norm_up = np.trace(gamma[:k,:k])
    norm_dn = np.trace(gamma[k:,k:])
    num = np.trace(gamma @ rdm1_fci)
    denom = np.trace(gamma @ gamma)*tr_fci
    distance = num**2/denom
    results[label] = [f'{norm:.3f}', f'{norm_up:.3f}', f'{norm_dn:.3f}', f'{distance:.3f}']

df2 = pd.DataFrame(results, index=['Tr_gamma', 'Tr_gamma_a', 'Tr_gamma_b', 'Distance'])

In [20]:
gamma = np.einsum('pqrq->pr', rdm2_fci) / (sum(nparts)-1)
norm = np.trace(gamma) 
norm_up = np.trace(gamma[:k,:k])
norm_dn = np.trace(gamma[k:,k:])
distance_fci = tr_fci**2/tr_fci**2
df2['FCI'] = [f'{norm:.3f}', f'{norm_up:.3f}', f'{norm_dn:.3f}', f'{distance_fci:.3f}']

In [21]:
df2

,hh2asymmtrnct,hh2lowdintrnct,hh2lowdinimgtrnct,FCI
Tr_gamma,4.000,4.000,4.000,4.000
Tr_gamma_a,2.000,2.000,2.000,2.000
Tr_gamma_b,2.000,2.000,2.000,2.000
Distance,0.996,0.996,0.996,1.000
